<a href="https://colab.research.google.com/github/GochiStuff/media-AI/blob/main/movieRecSystemV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval


In [10]:

path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("path to data files: " , path)

path to data files:  /root/.cache/kagglehub/datasets/tmdb/tmdb-movie-metadata/versions/2


Lets first gather some data about movies ( starting of with movies only ) and start building some ML model.

Using TMDB 5000 Movies dataset.

Types of Recommendation Systems

Generally , there are three types of such system .

1. Demographic Filter: the recommendation are same for every user despite their interest.
used for top trending , etc.

2. content-based filteration : Filter are based on movie meta data. system will recommend movies so the user will like them .

3. collaboration-based filteration : Here the system group users with similar interests and recommends movies to them // BRO THIS IS WHAT I THOUGHT . I thought it was my orignal Idea XC.


what I want to achieve .

In [7]:
print("YASH is going to build a super awesome AI that no one can beat !! ")

YASH is going to build a super awesome AI that no one can beat !! 


What I want to build .

1. A system that understand its user emotions and mood.
  Allow users to input their current mood manually.
  Use AI-based sentiment analysis (e.g., detect mood from text input or voice).

2. Context-Aware Recommendations
3. Hybrid Recommendation Engine
Combining Methods: Fuse collaborative filtering, content-based methods, and demographic insights with emotion and mood analysis. This creates a richer, more personalized profile.
Adaptive Learning: Implement reinforcement learning so the system improves its recommendations based on user interactions over time.

4. Explainable AI
Transparency: Provide users with insights on why a movie was recommended—be it due to their current mood, past behavior, or trending patterns among similar users.
Trust Building: This feature increases trust and lets users tweak parameters if they feel a recommendation doesn’t match their current taste.

5. Private . Version wise update the ai , else locally process.

... etc etc as I think of


but for Now I am only working on ver 1 Beta

In [25]:
credits_df = pd.read_csv(path + "/tmdb_5000_credits.csv")
movies_df = pd.read_csv(path + "/tmdb_5000_movies.csv")

print(movies_df.loc[1])

budget                                                          300000000
genres                  [{"id": 12, "name": "Adventure"}, {"id": 14, "...
homepage                     http://disney.go.com/disneypictures/pirates/
id                                                                    285
keywords                [{"id": 270, "name": "ocean"}, {"id": 726, "na...
original_language                                                      en
original_title                   Pirates of the Caribbean: At World's End
overview                Captain Barbossa, long believed to be dead, ha...
popularity                                                     139.082615
production_companies    [{"name": "Walt Disney Pictures", "id": 2}, {"...
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2007-05-19
revenue                                                         961000000
runtime                               

Now merge both the datasets into the movie data frame and view it.

In [26]:
credits_df.columns = ['id','title','cast','crew']
movies_df = movies_df.merge(credits_df, on='id')
print(movies_df.head())

      budget  ...                                               crew
0  237000000  ...  [{"credit_id": "52fe48009251416c750aca23", "de...
1  300000000  ...  [{"credit_id": "52fe4232c3a36847f800b579", "de...
2  245000000  ...  [{"credit_id": "54805967c3a36829b5002c41", "de...
3  250000000  ...  [{"credit_id": "52fe4781c3a36847f81398c3", "de...
4  260000000  ...  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

[5 rows x 23 columns]


# Start Building VB01

Demographic Filteration

In [32]:
# Calculate the mean vote average and 90th percentile vote count
c = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.9)

print(c)  # 6.092171559442016
print(m)  # 1838.4000000000015

# Keep only movies with vote_count >= m
new_movies_df = movies_df.loc[movies_df["vote_count"] >= m].copy()

print(new_movies_df.shape)  # (481, 23)

# Rename title column if needed
if "title_x" in new_movies_df.columns:
    new_movies_df.rename(columns={"title_x": "title"}, inplace=True)
elif "title_y" in new_movies_df.columns:
    new_movies_df.rename(columns={"title_y": "title"}, inplace=True)

# Define weighted rating function
def weighted_rating(x, m=m, c=c):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v + m) * R) + (m / (m + v) * c)

# Apply weighted rating and sort movies
new_movies_df['score'] = new_movies_df.apply(weighted_rating, axis=1)
new_movies_df = new_movies_df.sort_values('score', ascending=False)

# Display top 10 movies
print(new_movies_df[['title', 'vote_count', 'vote_average', 'score']].head(10))


6.092171559442016
1838.4000000000015
(481, 23)
Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')
                                              title  vote_count  vote_average     score
1881                       The Shawshank Redemption        8205           8.5  8.059258
662                                      Fight Club        9413           8.3  7.939256
65                                  The Dark Knight       12002           8.2  7.920020
3232                                   Pulp Fiction        8428           8.3  7.904645
96                                        Inception       13752           8.1  7.863239
3337                                  The Godfather      

content - based Filtering


In [45]:
# Step 1: Load and Preprocess Overview
movies_df["overview"] = movies_df["overview"].fillna("")

# Step 2: Apply TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies_df["overview"])
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)  # Example: (4800, 20000)

# Step 3: Compute Cosine Similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print("Cosine Similarity Matrix Shape:", cosine_sim.shape)  # Example: (4800, 4800)

# Step 4: Create a Reverse Lookup Table for Movie Titles
indices = pd.Series(movies_df.index, index=movies_df["title_x"]).drop_duplicates()
print(indices.head(5))


TF-IDF Matrix Shape: (4803, 20978)
Cosine Similarity Matrix Shape: (4803, 4803)
title_x
Avatar                                      0
Pirates of the Caribbean: At World's End    1
Spectre                                     2
The Dark Knight Rises                       3
John Carter                                 4
dtype: int64


In [71]:
from difflib import get_close_matches

def get_recommendations(title, cosine_sim=cosine_sim, movies_df=movies_df, indices=indices):
    # Step 1: Get the index of the movie that matches the title
    if title not in indices:
        close_matches = get_close_matches(title, list(indices.index), n=6, cutoff=0.6)
        if close_matches:
            return "Movie not found in dataset. Did you mean: " + ", ".join(close_matches) + "?"
        else:
            return "Movie not found in dataset. Try another title."

    idx = indices[title]

    # Step 2: Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Step 3: Sort the movies based on similarity scores (Descending order)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Step 4: Get the top 10 most similar movies (excluding the input movie)
    sim_scores = sim_scores[1:11]

    # Step 5: Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Step 6: Return the top 10 similar movies
    return movies_df["title_x"].iloc[movie_indices]

# Example Usage:
finder = input("What movie ?? : ")
print(get_recommendations(finder))


What movie ?? : Begin
Movie not found in dataset. Did you mean: Legion, Beginners, Begin Again, Megamind, Benji?
